# Objetivo 

<b>Sobre o conjunto de dados:</b> Este dataset foi fornecido pela Olist que tem como objetivo conecta pequenas empresas de todo o Brasil a canais sem complicações e com um único contrato. Nele contém informações de 100 mil pedidos de 2016 a 2018 realizados em diversos mercados no Brasil. 

O objetivo deste notebook é

- Carregar as tabelas para um banco de dados. 
- Responder algumas questões sobre o conjunto de dados utilizando a linguagem SQL.



 

# 0.0 Imports

In [1]:
import re
import requests
import warnings
import inflection

import numpy   as np
import pandas  as pd
import seaborn as sns

from datetime              import datetime
from bs4                   import BeautifulSoup
from IPython.core.display  import HTML
from IPython.display       import Image

from matplotlib            import pyplot as plt

## 0.1 Helper Functions

In [2]:
def jupyter_settings():
    
    %matplotlib inline 
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[20,10]
    plt.rcParams['font.size']=10
    
    display( HTML('<style>.container {width:90% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    #pd.set_options('display.expand_frame_repr',False )
    
    sns.set()

In [3]:
jupyter_settings()

warnings.filterwarnings ('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/bruna/.pyenv/versions/3.8.0/envs/datascienceemproducao/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# 1.0 Connection Database

In [4]:
import sqlite3
from sqlalchemy import create_engine

In [5]:
db = create_engine('sqlite:///db_olist.sqlite', echo = False)

conn = db.connect()

# 2.0 Loading Dataset

In [25]:
df_orders = pd.read_csv('../datasets/olist_orders_dataset.csv')
df_sellers = pd.read_csv('../datasets/olist_sellers_dataset.csv')
df_products = pd.read_csv('../datasets/olist_products_dataset.csv')
df_items = pd.read_csv('../datasets/olist_order_items_dataset.csv')
df_customers = pd.read_csv('../datasets/olist_customers_dataset.csv')
df_reviews = pd.read_csv('../datasets/olist_order_reviews_dataset.csv')
df_geolocation = pd.read_csv('../datasets/olist_geolocation_dataset.csv')
df_payments = pd.read_csv('../datasets/olist_order_payments_dataset.csv')
df_category = pd.read_csv('../datasets/product_category_name_translation.csv')

# 3.0 Carregando Dataset in the database

## 3.1 Table Orders

In [18]:
schema_orders= '''

CREATE TABLE tb_orders(
        order_id                         TEXT,
        customer_id                      TEXT,
        order_status                     TEXT,
        order_purchase_timestamp         TEXT,
        order_approved_at                TEXT,
        order_delivered_carrier_date     TEXT,
        order_delivered_customer_date    TEXT,
        order_estimated_delivery_date    TEXT

)
'''

#create schema
#conn.execute(schema_orders)

#insert data into database
df_orders.to_sql('orders', con=conn, if_exists='append', index=False)


## 3.2 Table Sellers

In [30]:
schema_sellers= '''

CREATE TABLE tb_sellers(
        seller_id                 TEXT,
        seller_zip_code_prefix    INTEGER,
        seller_city               TEXT,
        seller_state              TEXT

)
'''

#create schema
#conn.execute(schema_sellers)

#insert data into database
df_sellers.to_sql('sellers', con=conn, if_exists='append', index=False)

## 3.3 Table Products

In [33]:
schema_products= '''

CREATE TABLE tb_products(
        product_id                    TEXT,
        product_category_name         TEXT,
        product_name_lenght           REAL,
        product_description_lenght    REAL,
        product_photos_qty            REAL,
        product_weight_g              REAL,
        product_length_cm             REAL,
        product_height_cm             REAL,
        product_width_cm              REAL

)
'''

#create schema
conn.execute(schema_products)

#insert data into database
df_products.to_sql('products', con=conn, if_exists='append', index=False)

## 3.4 Table Items

In [35]:
schema_items= '''

CREATE TABLE tb_items(
        order_id                TEXT,
        order_item_id           INTEGER,
        product_id              TEXT,
        seller_id               TEXT,
        shipping_limit_date     TEXT,
        price                   REAL,
        freight_value           REAL

)
'''

#create schema
conn.execute(schema_items)

#insert data into database
df_items.to_sql('items', con=conn, if_exists='append', index=False)

## 3.5 Table Customers

In [ ]:
schema_customers= '''

CREATE TABLE tb_customers(
        customer_id                 TEXT,
        customer_unique_id          TEXT,
        customer_zip_code_prefix    INTEGER,
        customer_city               TEXT,
        customer_state              TEXT
)
'''

#create schema
conn.execute(schema_customers)

#insert data into database
df_customers.to_sql('customers', con=conn, if_exists='append', index=False)

## 3.6 Table Reviews

In [40]:
schema_reviews= '''

CREATE TABLE tb_reviews(
        review_id                  TEXT,
        order_id                   TEXT,
        review_score               INTEGER,
        review_comment_title       TEXT,
        review_comment_message     TEXT,
        review_creation_date       TEXT,
        review_answer_timestamp    TEXT
)
'''

#create schema
conn.execute(schema_reviews)

#insert data into database
df_reviews.to_sql('reviews', con=conn, if_exists='append', index=False)

## 3.7 Table Geolocation

In [42]:
schema_geolocation= '''

CREATE TABLE tb_geolocation(
        geolocation_zip_code_prefix     INTEGER,
        geolocation_lat                 REAL,
        geolocation_lng                 REAL,
        geolocation_city                TEXT,
        geolocation_state               TEXT
)
'''

#create schema
conn.execute(schema_geolocation)

#insert data into database
df_geolocation.to_sql('geolocation', con=conn, if_exists='append', index=False)

## 3.8 Table Payments

In [45]:
schema_payments= '''

CREATE TABLE tb_payments(
        order_id                 TEXT,
        payment_sequential       INTEGER, 
        payment_type             TEXT,
        payment_installments     INTEGER,
        payment_value            REAL
)
'''

#create schema
conn.execute(schema_payments)

#insert data into database
df_payments.to_sql('payments', con=conn, if_exists='append', index=False)

## 3.9 Table Category

In [47]:
schema_category= '''

CREATE TABLE tb_category(
        product_category_name            TEXT,
        product_category_name_english    TEXT
)
'''

#create schema
conn.execute(schema_category)

#insert data into database
df_category.to_sql('category', con=conn, if_exists='append', index=False)

# 3.10 Query

In [18]:
# verificar as tabelas que possuo no banco de dados
query = '''
    SELECT name
    FROM sqlite_master
    WHERE type = 'table'
'''

# query = '''
#     SELECT *
#     FROM sellers
# '''

# query = '''
#     ALTER TABLE category
#     RENAME TO tb_category;

# '''
table = pd.read_sql_query(query,conn)
table

,name
0,tb_orders
1,tb_sellers
2,tb_products
3,tb_items
4,tb_customers
5,tb_reviews
6,tb_geolocation
7,tb_payments
8,tb_category


# 4.0 Exercícios Propostos

## 4.1 Quantos pedidos foram feitos por cada tipo de pagamento?

In [6]:
query = '''
select 
    p.payment_type,	
    count(o.order_id) as type_payments
from tb_orders o inner join tb_payments p ON (p.order_id = o.order_id)
group by p.payment_type
order by type_payments DESC;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,payment_type,type_payments
0,credit_card,76795
1,boleto,19784
2,voucher,5775
3,debit_card,1529
4,not_defined,3


## 4.2 Qual o número máximo e mínimo de parcelas feitas nos pagamentos?

In [7]:
query = '''
select 
    min(p.payment_installments) as min_parcelas,
    max(p.payment_installments) as max_parcelas
from tb_payments p;'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,min_parcelas,max_parcelas
0,0,24


## 4.3 Quais são os Top 10 pedidos com os maiores valores de pagamento?

In [8]:
query = '''
select
    p.order_id,
    p.payment_value 
from tb_payments p
order by p.payment_value DESC
limit 10;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,order_id,payment_value
0,03caa2c082116e1d31e67e9ae3700499,13664.08
1,736e1922ae60d0d6a89247b851902527,7274.88
2,0812eb902a67711a1cb742b3cdaa65ae,6929.31
3,fefacc66af859508bf1a7934eab1e97f,6922.21
4,f5136e38d1a14a4dbd87dff67da82701,6726.66
5,2cc9089445046817a7539d90805e6e5a,6081.54
6,a96610ab360d42a2e5335a3998b4718a,4950.34
7,b4c4b76c642808cbe472a32b86cddc95,4809.44
8,199af31afc78c699f0dbf71fb178d4d4,4764.34
9,8dbc85d1447242f3b127dda390d56e19,4681.78


## 4.4 Quais são os últimos 10 pedidos com os menores valores de pagamentos?

In [9]:
query = '''
select 
    p.order_id,
    p.payment_value
from tb_orders o inner join tb_payments p ON (p.order_id = o.order_id)
where p.payment_value >=1
order by p.payment_value ASC
limit 10;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,order_id,payment_value
0,285c2e15bebd4ac83635ccc563dc71f4,1.00
1,d80faf302bbccea1f0b613d440de4d33,1.01
2,fbd05af8c1751b972fc6e19b1190990e,1.02
3,065c05543ebdfd79fb629b2b4a6ab904,1.02
4,285c2e15bebd4ac83635ccc563dc71f4,1.03
5,ccf804e764ed5650cd8759557269dc13,1.03
6,1fb40b224a9509697bacc23d3e160e4c,1.03
7,6d58638e32674bebee793a47ac4cbadc,1.03
8,8ee8450594de505503ee786c1a401800,1.04
9,ccf804e764ed5650cd8759557269dc13,1.04


## 4.5 Qual a média do valor de pagamento por tipo?

In [10]:
query = '''
select 
    p.payment_type,
    AVG(p.payment_value) AS avg_type_payments
from tb_payments p
group by p.payment_type
order by p.payment_value DESC;
'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,payment_type,avg_type_payments
0,debit_card,142.570170
1,credit_card,163.319021
2,boleto,145.034435
3,voucher,65.703354
4,not_defined,0.000000


## 4.6 Quais os Top 5 clientes com os maiores valores de pagamentos no boleto?

In [11]:
query = '''
select 
    o.customer_id,
    p.payment_type,
    p.payment_value
from tb_orders o inner join tb_payments p ON (p.order_id = o.order_id)
where p.payment_type == 'boleto'
group by o.customer_id
order by p.payment_value DESC
limit 5;
'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,customer_id,payment_type,payment_value
0,ec5b2ba62e574342386871631fafd3fc,boleto,7274.88
1,f48d464a0baaea338cb25f816991ab1f,boleto,6922.21
2,3fd6777bbce08a352fddd04e4a7cc8f6,boleto,6726.66
3,05455dfa7cd02f13d132aa7a6a9729c6,boleto,6081.54
4,31e83c01fce824d0ff786fcd48dad009,boleto,3979.55


## 4.7 Quais os Top 5 clientes com os maiores valores de pagamento no cartão de crédito?

In [12]:
query = '''
select 
    o.customer_id,
    p.payment_type,
    p.payment_value
from tb_orders o inner join tb_payments p ON (p.order_id = o.order_id)
where p.payment_type == 'credit_card'
group by o.customer_id
order by p.payment_value DESC
limit 5;
'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,customer_id,payment_type,payment_value
0,1617b1357756262bfa56ab541c47bc16,credit_card,13664.08
1,c6e2731c5b391845f6800c97401a43a9,credit_card,6929.31
2,df55c14d1476a9a3467f131269c2477f,credit_card,4950.34
3,e0a2412720e9ea4f26c1ac985f6a7358,credit_card,4809.44
4,24bbf5fd2f2e1b359ee7de94defc4a15,credit_card,4764.34


## 4.8 Quais os top 10 produtos mais caro?

In [13]:
#Usando o group by
query = ''' 
select 
    po.product_id,
    po.product_category_name,
    i.price
from tb_items i inner join tb_products po ON (po.product_id = i.product_id) 
group by po.product_id
order by i.price DESC
limit 10;'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,product_id,product_category_name,price
0,489ae2aa008f021502940f251d4cce7f,utilidades_domesticas,6735.00
1,69c590f7ffc7bf8db97190b6cb6ed62e,pcs,6729.00
2,1bdf5e6731585cf01aa8169c7028d6ad,artes,6499.00
3,a6492cc69376c469ab6f61d8f44de961,eletroportateis,4799.00
4,c3ed642d592594bb648ff4a04cee2747,eletroportateis,4690.00
5,259037a6a41845e455183f89c5035f18,pcs,4590.00
6,a1beef8f3992dbd4cd8726796aa69c53,instrumentos_musicais,4399.87
7,6cdf8fc1d741c76586d8b6b15e9eef30,consoles_games,4099.99
8,6902c1962dd19d540807d0ab8fade5c6,relogios_presentes,3999.90
9,4ca7b91a31637bd24fb8e559d5e015e4,eletroportateis,3999.00


In [14]:
#Usando o Distinct

query = '''

select 
    DISTINCT(po.product_id),
    po.product_category_name,
    i.price
from tb_items i inner join tb_products po ON (po.product_id = i.product_id) 
order by i.price DESC
limit 10;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,product_id,product_category_name,price
0,489ae2aa008f021502940f251d4cce7f,utilidades_domesticas,6735.00
1,69c590f7ffc7bf8db97190b6cb6ed62e,pcs,6729.00
2,1bdf5e6731585cf01aa8169c7028d6ad,artes,6499.00
3,a6492cc69376c469ab6f61d8f44de961,eletroportateis,4799.00
4,c3ed642d592594bb648ff4a04cee2747,eletroportateis,4690.00
5,259037a6a41845e455183f89c5035f18,pcs,4590.00
6,a1beef8f3992dbd4cd8726796aa69c53,instrumentos_musicais,4399.87
7,6cdf8fc1d741c76586d8b6b15e9eef30,consoles_games,4099.99
8,dd113cb02b2af9c8e5787e8f1f0722f6,esporte_lazer,4059.00
9,6902c1962dd19d540807d0ab8fade5c6,relogios_presentes,3999.90


## 4.9 Quais os top 10 produtos mais baratos?

In [15]:
query = '''
select 
    po.product_id,
    po.product_category_name,
    i.price
from tb_items i inner join tb_products po ON (po.product_id = i.product_id)
group by po.product_id
order by i.price ASC
limit 10;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,product_id,product_category_name,price
0,8a3254bee785a526d548a81a9bc3c9be,construcao_ferramentas_construcao,0.85
1,05b515fdc76e888aada3c6d66c201dff,beleza_saude,1.20
2,270516a3f41dc035aa87d220228f844c,beleza_saude,1.20
3,46fce52cef5caa7cc225a5531c946c8b,beleza_saude,2.20
4,310dc32058903b6416c71faff132df9e,papelaria,2.29
5,680cc8535be7cc69544238c1d6a83fe8,pet_shop,2.90
6,2e8316b31db34314f393806fd7b6e185,papelaria,2.99
7,44d53f1240d6332232e4393c06500475,None,3.00
8,29781581fb82fe2389560a3a5331d0ee,utilidades_domesticas,3.06
9,1716ea399ed8ee62ba811e6f55180f45,automotivo,3.49


## 4.10 Quais as Top 10 categorias mais compradas? 

In [16]:
query = '''
select 
    po.product_category_name,
    COUNT(o.order_id) AS qt_category
from tb_items i inner join tb_products po ON (po.product_id = i.product_id) 
inner join tb_orders o ON (o.order_id = i.order_id)
where po.product_category_name != 'NULL'
group by po.product_category_name
order by qt_category DESC
limit 10;

'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,product_category_name,qt_category
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
5,utilidades_domesticas,6964
6,relogios_presentes,5991
7,telefonia,4545
8,ferramentas_jardim,4347
9,automotivo,4235


## 4.11 Quais são os Top 5 produtos com o maior número de review?

In [17]:
query = '''
WITH products_total AS (
    select 
        product_id
    from tb_products po

), item_total AS(
    select 
        order_id,
        product_id
    from tb_items i
    
), ordem_total AS(
    select 
        order_id
    from tb_orders o

),review_total AS(
    select *
    from tb_reviews r
)
select 
    pt.product_id,
    SUM(rt.review_score) AS review_sum
from products_total pt inner join item_total it ON (it.product_id = pt.product_id) 
                       inner join ordem_total ot ON (ot.order_id = it.order_id)
                       inner join review_total rt ON (rt.order_id = ot.order_id)
group by pt.product_id 
order by review_sum DESC
limit 10;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,product_id,review_sum
0,aca2eb7d00ea1a7b8ebd4e68314663af,2112
1,422879e10f46682990de24d770e7f83d,1920
2,99a4788cb24856965c36a24e339b6058,1896
3,389d119b48cf3043d311335e499d9c6b,1612
4,368c6c730842d78016ad823897a372db,1531
5,53759a2ecddad2bb87a079a1f1519f73,1448
6,d1c427060a0f73f6b889a5c7c61f2ac4,1432
7,53b36df67ebb7c41585e8d54d6772e08,1348
8,154e7e31ebfa092203795c972e5804a6,1264
9,3dd2a17168ec895c781a9191c1e95ad7,1147


## 4.12 Quais são os Top 10 produtos sem nenhum review?

In [18]:
query = '''
WITH products_total AS (
    select 
        po.product_id
    from tb_products po

), item_total AS(
    select 
        i.order_id,
        i.product_id
    from tb_items i
        
), ordem_total AS(
    select 
        o.order_id
    from tb_orders o

),review_total AS(
    select *
    from tb_reviews r
)
select 
    pt.product_id,
    SUM(rt.review_score) AS review_sum
from products_total pt inner join item_total it ON (it.product_id = pt.product_id) 
                       inner join ordem_total ot ON (ot.order_id = it.order_id)
                       inner join review_total rt ON (rt.order_id = ot.order_id)
group by pt.product_id 
order by review_sum 
limit 10;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,product_id,review_sum
0,0009406fd7479715e4bef61dd91f2462,1
1,0011c512eb256aa0dbbb544d8dffcf6e,1
2,002d4ea7c04739c130bb74d7e7cd1694,1
3,0043c62d00db47eff6a6bc4cf6bfaeda,1
4,0043d1a25ef08fb6f41b8fa6f91742ab,1
5,0060b415594c5e1200324ef1a18493c4,1
6,00cf06147ed4880ec5fbba2adbb20e1d,1
7,00e62bcf9337ca4c5d5b4c5c8188f8d2,1
8,00efb9a8ce0cf3b2f37892ab003edc10,1
9,0103863bf3441460142ec23c74388e4c,1


## 4.13 Quais são os Top 10 clientes com a maior quantidade de pedidos?

In [19]:
query = '''select 
    o.customer_id,
    COUNT(i.order_item_id) AS qtd_pedidos
from tb_orders o inner join tb_items i ON (i.order_id = o.order_id)
group by o.customer_id 
order by qtd_pedidos DESC
limit 10;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,customer_id,qtd_pedidos
0,fc3d1daec319d62d49bfb5e1f83123e9,21
1,be1b70680b9f9694d8c70f41fa3dc92b,20
2,bd5d39761aa56689a265d95d8d32b8be,20
3,adb32467ecc74b53576d9d13a5a55891,15
4,10de381f8a8d23fff822753305f71cae,15
5,d5f2b3f597c7ccafbb5cac0bcc3d6024,14
6,a7693fba2ff9583c78751f2b66ecab9d,14
7,7d321bd4e8ba1caf74c4c1aabd9ae524,13
8,daf15f1b940cc6a72ba558f093dc00dd,12
9,9eb3d566e87289dcb0acf28e1407c839,12


## 4.14 Quais são os 10 clientes com a menor quantidade de pedidos?

In [20]:
query= '''
select 
    o.customer_id,
    COUNT(i.order_item_id) AS qtd_pedidos
from tb_orders o inner join tb_items i ON (i.order_id = o.order_id)
group by o.customer_id 
order by qtd_pedidos 
limit 10;

'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,customer_id,qtd_pedidos
0,00012a2ce6f8dcda20d059ce98491703,1
1,000161a058600d5901f007fab4c27140,1
2,0001fd6190edaaf884bcaf3d49edf079,1
3,0002414f95344307404f0ace7a26f1d5,1
4,000379cdec625522490c315e70c7a9fb,1
5,0004164d20a9e969af783496f3408652,1
6,000419c5494106c306a97b5635748086,1
7,00046a560d407e99b969756e0b10f282,1
8,00050bf6e01e69d5c0fd612f1bcfb69c,1
9,000598caf2ef4117407665ac33275130,1


## 4.15 Quais vendedores existem na base?

In [21]:
query = '''
select 
    seller_id
from tb_sellers
limit 10;
'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,seller_id
0,3442f8959a84dea7ee197c632cb2df15
1,d1b65fc7debc3361ea86b5f14c68d2e2
2,ce3ad9de960102d0677a81f5d0bb7b2d
3,c0f3eea2e14555b6faeea3dd58c1b1c3
4,51a04a8a6bdcb23deccc82b0b80742cf
5,c240c4061717ac1806ae6ee72be3533b
6,e49c26c3edfa46d227d5121a6b6e4d37
7,1b938a7ec6ac5061a66a3766e0e75f90
8,768a86e36ad6aae3d03ee3c6433d61df
9,ccc4bbb5f32a6ab2b7066a4130f114e3


## 4.16  Qual a distribuição de vendedores por estado?

In [22]:
query = '''

select 
    s.seller_state,
    COUNT(s.seller_id) AS qtd_vendedores
from tb_sellers s
group by s.seller_state
order by qtd_vendedores DESC;
'''
query_execute = pd.read_sql_query(query,conn)
query_execute

,seller_state,qtd_vendedores
0,SP,1849
1,PR,349
2,MG,244
3,SC,190
4,RJ,171
5,RS,129
6,GO,40
7,DF,30
8,ES,23
9,BA,19


## 4.17 Qual a distribuição de clientes por estado?

In [24]:
query = '''

select 
    c.customer_state,
    COUNT(c.customer_id) AS qtd_clientes
from tb_customers c
group by c.customer_state
order by qtd_clientes DESC
'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,customer_state,qtd_clientes
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045
5,SC,3637
6,BA,3380
7,DF,2140
8,ES,2033
9,GO,2020


## 4.18 Quais são os top 10 vendedores, em termos de números de vendas?

In [25]:
query = '''
WITH sb_sellers AS (
    select
        s.seller_id
    from tb_sellers s
    
), sb_items AS (
    select
        i.order_id,
        i.seller_id
    from tb_items i
    
), sb_order AS (
    select
        o.order_id
    from tb_orders o

)
select 
    vs.seller_id,
    COUNT(vo.order_id) AS qtd_sell
from sb_sellers vs inner join sb_items vi ON (vi.seller_id = vs.seller_id)
                   inner join sb_order vo ON (vo.order_id = vi.order_id)
group by vs.seller_id
order by qtd_sell DESC
limit 10;

'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,seller_id,qtd_sell
0,6560211a19b47992c3666cc44a7e94c0,2033
1,4a3ca9315b744ce9f8e9374361493884,1987
2,1f50f920176fa81dab994f9023523100,1931
3,cc419e0650a3c5ba77189a1882b7556a,1775
4,da8622b14eb17ae2831f4ac5b9dab84a,1551
5,955fee9216a65b617aa5c0531780ce60,1499
6,1025f0e2d44d7041d6cf58b6550e0bfa,1428
7,7c67e1448b00f6e969d365cea6b010ab,1364
8,ea8482cd71df3c1969d7b9473ff13abc,1203
9,7a67c85e85bb2ce8582c35f2203ad736,1171


## 4.19 Quais são os 10 piores vendedores, em termos de número de vendas?

In [26]:
query = '''
WITH sb_sellers AS (
    select
        s.seller_id
    from tb_sellers s
    
), sb_items AS (
    select
        i.order_id,
        i.seller_id
    from tb_items i
    
), sb_order AS (
    select
        o.order_id
    from tb_orders o

)
select 
    vs.seller_id,
    COUNT(vo.order_id) AS qtd_sell
from sb_sellers vs inner join sb_items vi ON (vi.seller_id = vs.seller_id)
                   inner join sb_order vo ON (vo.order_id = vi.order_id)
group by vs.seller_id
order by qtd_sell 
limit 10;

'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,seller_id,qtd_sell
0,001e6ad469a905060d959994f1b41e4f,1
1,003554e2dce176b5555353e4f3555ac8,1
2,00ab3eff1b5192e5f1a63bcecfee11c8,1
3,00d8b143d12632bad99c0ad66ad52825,1
4,04ee0ec01589969663ba5967c0e0bdc0,1
5,05a48cc8859962767935ab9087417fbb,1
6,05ca864204d09595ae591b93ea9cf93d,1
7,07a9bc7f61396b658ddf3ac534efe44c,1
8,08d1c9478fb0d7d7fcac9c0da4e82fc7,1
9,0a198e95d32b1be2da9424c962a6ebfa,1


## 4.20- Quantos produtos são comprados, em média, por pedido de compra?

In [27]:
query = '''
select 
    o.order_id,
    AVG(i.order_item_id) AS average
from tb_items i inner join tb_orders o ON (o.order_id = i.order_id)
group by o.order_id
order by average DESC
limit 10;

'''

query_execute = pd.read_sql_query(query,conn)
query_execute

,order_id,average
0,8272b63d03f5f79c56e9e4120aec44ef,11.0
1,ab14fdcfbe524636d65ee38360e22ce8,10.5
2,1b15974a0141d54e36626dca3fdc731a,10.5
3,9ef13efd6949e4573a18964dd1bbe7f5,8.0
4,428a2f660dc84138d969ccd69a0ab6d5,8.0
5,9bdc4d4c71aa1de4606060929dee888c,7.5
6,73c8ab38f07dc94389065f7eba4f297a,7.5
7,37ee401157a3a0b28c9c6d0ed8c3b24b,7.0
8,c05d6a79e55da72ca780ce90364abed9,6.5
9,af822dacd6f5cff7376413c03a388bb7,6.5
